In [1]:
import os
import copy
import warnings

import torch
import random
import pandas as pd
import numpy  as np
import torch.nn as nn
from src.processing import *
from torchvision import models
from torch.utils.data import DataLoader
import src.model_fitting as model_fitting

warnings.filterwarnings("ignore")

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = True

C:\Users\ozher\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
images_amount = check_images_amount()

annotations = pd.read_csv("annotation_file.csv")
annot_test = pd.read_csv("annotation_file_test.csv").sort_values(by="id")

NEW_SIZE, BATCH_SIZE = 224, 32
full_train = CarDataset(annotation_file=annotations, img_dir="train", new_size=NEW_SIZE, mode="train")
full_train_dataloader = DataLoader(full_train, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = CarDataset(annotation_file=annot_test, img_dir="public_test", new_size=NEW_SIZE, mode="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [3]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 11)
model = model.cuda()

torch.cuda.empty_cache()

In [4]:
fitting_info = model_fitting.train(full_train_dataloader, None, model=model, epochs=10, validate=False)

torch.save(copy.deepcopy(model.state_dict()), 'resnet18_fitting_weights.pth')
torch.cuda.empty_cache()

epoch:  10%|████████████████▋                                                                                                                                                      | 1/10 [00:42<06:18, 42.05s/it]

[0]: tr_loss: 0.48236 tr_f1: 0.83502 


epoch:  20%|█████████████████████████████████▍                                                                                                                                     | 2/10 [01:21<05:25, 40.70s/it]

[1]: tr_loss: 0.16226 tr_f1: 0.94533 


epoch:  30%|██████████████████████████████████████████████████                                                                                                                     | 3/10 [02:01<04:42, 40.40s/it]

...

[7]: tr_loss: 0.01036 tr_f1: 0.99664 


epoch:  90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                | 9/10 [06:00<00:39, 39.89s/it]

[8]: tr_loss: 0.01215 tr_f1: 0.99659 


epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [06:40<00:00, 40.01s/it]

[9]: tr_loss: 0.00583 tr_f1: 0.99846 


In [6]:
inverse_target_mapping = {path_idx: path for path_idx, path in enumerate(os.listdir("train"))} 

preds = model_fitting.predict(model, test_dataloader, inverse_target_mapping=inverse_target_mapping)
if((len(preds) == images_amount["test"]) and (len(np.unique(preds)) == 11)):
    make_submit("submit", preds)

File 'submit_new.csv' already exist. Are you want to rewrite it? [yes/no]
>no
Define new name
>000
=====SUBMITION DONE=====


---